In [2]:
#Preprocessing data to apply collaborative filtering to generate user id - movieid matrix and find k nearest neighbours for a test user
# import libraries 

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
movies = pd.read_csv('datasets/movies.csv')

In [7]:
movies.shape

(9742, 3)

In [9]:
movies.columns

Index(['movieId', 'title', 'genres'], dtype='object')

In [10]:
movies.head(10)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [11]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [12]:
movies['genres'] = movies['genres'].str.replace('|', ' ')

C:\Users\FATIMA\AppData\Local\Temp\ipykernel_11332\4181581663.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  movies['genres'] = movies['genres'].str.replace('|', ' ')


In [13]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure Animation Children Comedy Fantasy
1,2,Jumanji (1995),Adventure Children Fantasy
2,3,Grumpier Old Men (1995),Comedy Romance
3,4,Waiting to Exhale (1995),Comedy Drama Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action Animation Comedy Fantasy
9738,193583,No Game No Life: Zero (2017),Animation Comedy Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action Animation


In [14]:
movies['title'] = movies['title'].str.lower()

In [20]:
movies['title'] = movies['title'].apply(lambda x : x[:-6])

In [21]:
movies

,movieId,title,genres
0,1,toy story,Adventure Animation Children Comedy Fantasy
1,2,jumanji,Adventure Children Fantasy
2,3,grumpier old men,Comedy Romance
3,4,waiting to exhale,Comedy Drama Romance
4,5,father of the bride part ii,Comedy
...,...,...,...
9737,193581,black butler: book of the atlantic,Action Animation Comedy Fantasy
9738,193583,no game no life: zero,Animation Comedy Fantasy
9739,193585,flint,Drama
9740,193587,bungo stray dogs: dead apple,Action Animation


In [24]:
ratings = pd.read_csv('datasets/ratings.csv')

In [25]:
ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [26]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [27]:
ratings.tail()

,userId,movieId,rating,timestamp
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352
100835,610,170875,3.0,1493846415


In [28]:
ratings.shape

(100836, 4)

In [29]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [30]:
from sklearn.model_selection import train_test_split

X_train, X_test = train_test_split(ratings, test_size = 0.30, random_state = 42)

print(X_train.shape)
print(X_test.shape)

(70585, 4)
(30251, 4)


In [31]:
# pivot ratings matrix into movie features matrix
user_data = X_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(0)
user_data.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193571,193573,193579,193581,193583,193585,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
# make a copy of train and test datasets
dummy_train = X_train.copy()
dummy_test = X_test.copy()

dummy_train['rating'] = dummy_train['rating'].apply(lambda x: 0 if x > 0 else 1)
dummy_test['rating'] = dummy_test['rating'].apply(lambda x: 1 if x > 0 else 0)

In [33]:
# The movies not rated by user is marked as 1 for prediction 
dummy_train = dummy_train.pivot(index = 'userId', columns = 'movieId', values = 'rating').fillna(1)

# The movies not rated by user is marked as 0 for evaluation 
dummy_test = dummy_test.pivot(index ='userId', columns = 'movieId', values = 'rating').fillna(0)

In [34]:
dummy_train.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193571,193573,193579,193581,193583,193585,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,1.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
5,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [35]:
dummy_test.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,187595,189043,189111,189333,189547,189713,190213,190219,193567,193587
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
from sklearn.metrics.pairwise import cosine_similarity

# User Similarity Matrix using Cosine similarity as a similarity measure between Users and precting k-nearest neighbours
user_similarity = cosine_similarity(user_data)
user_similarity[np.isnan(user_similarity)] = 0
print(user_similarity)
print(user_similarity.shape)

[[1.         0.01799262 0.02914591 ... 0.22586534 0.12184689 0.13241346]
 [0.01799262 1.         0.         ... 0.04299081 0.03472882 0.0625667 ]
 [0.02914591 0.         1.         ... 0.00378273 0.         0.02382229]
 ...
 [0.22586534 0.04299081 0.00378273 ... 1.         0.10982639 0.21962266]
 [0.12184689 0.03472882 0.         ... 0.10982639 1.         0.05416081]
 [0.13241346 0.0625667  0.02382229 ... 0.21962266 0.05416081 1.        ]]
(610, 610)


In [37]:
user_predicted_ratings = np.dot(user_similarity, user_data)


In [38]:
user_predicted_ratings

array([[8.52008912e+01, 3.99290227e+01, 2.08165173e+01, ...,
        2.81820351e-02, 2.81820351e-02, 1.57425084e-01],
       [2.45531356e+01, 1.05987273e+01, 2.94172315e+00, ...,
        8.74389309e-02, 8.74389309e-02, 4.49741734e-01],
       [4.22670774e+00, 2.11463396e+00, 9.59320674e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.49265311e+01, 4.92867089e+01, 2.12555470e+01, ...,
        2.51644930e-02, 2.51644930e-02, 5.92666313e-01],
       [7.50962548e+01, 3.56578151e+01, 1.17138113e+01, ...,
        0.00000000e+00, 0.00000000e+00, 6.43083908e-02],
       [7.67153155e+01, 3.67625117e+01, 1.11564580e+01, ...,
        2.61466866e-01, 2.61466866e-01, 8.03583319e-01]])

In [39]:
user_predicted_ratings.shape

(610, 8566)

In [40]:
# np.multiply for cell-by-cell multiplication 

user_final_ratings = np.multiply(user_predicted_ratings, dummy_train)
user_final_ratings.head()


movieId,1,2,3,4,5,6,7,8,9,10,...,190221,191005,193565,193571,193573,193579,193581,193583,193585,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000000,39.929023,0.000000,1.220732,10.483532,0.000000,14.082187,2.605186,4.111435,47.981999,...,0.083324,0.036234,0.028182,0.032208,0.032208,0.028182,0.032208,0.028182,0.028182,0.157425
2,24.553136,10.598727,2.941723,0.085473,2.773060,10.458197,2.308815,0.523106,0.588578,11.698251,...,0.054182,0.112421,0.087439,0.099930,0.099930,0.087439,0.099930,0.087439,0.087439,0.449742
3,4.226708,2.114634,0.959321,0.066644,0.308448,2.855740,0.518991,0.105484,0.190006,2.550762,...,0.008707,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,50.832807,23.786786,9.793001,0.706291,6.609481,24.594138,10.881808,1.505819,2.196956,27.970658,...,0.005910,0.065396,0.050864,0.058130,0.058130,0.050864,0.058130,0.050864,0.050864,0.106328
5,60.794262,36.875980,12.358414,2.321889,13.215004,29.323176,14.149193,3.043103,2.351687,47.983608,...,0.108995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.103332


In [41]:
user_final_ratings.iloc[42].sort_values(ascending = False)[0:5]

movieId
480    87.452973
150    86.994790
457    81.930003
593    80.807548
1      61.416804
Name: 43, dtype: float64

In [42]:
user_final_ratings.iloc[90].sort_values(ascending = False)[0:10]
user1 = user_final_ratings.iloc[90].sort_values(ascending = False)

In [43]:
user1.head()

movieId
296     145.821747
593     118.820510
260     114.589938
589     106.047278
1198    103.143962
Name: 91, dtype: float64

In [44]:
user1.shape

(8566,)

In [46]:
user1

movieId
296     145.821747
593     118.820510
260     114.589938
589     106.047278
1198    103.143962
           ...    
494       0.000000
3810      0.000000
7028      0.000000
500       0.000000
1584      0.000000
Name: 91, Length: 8566, dtype: float64

In [47]:
newuser= user1.to_frame()

In [48]:
newuser

,91
movieId,
296,145.821747
593,118.820510
260,114.589938
589,106.047278
1198,103.143962
...,...
494,0.000000
3810,0.000000
7028,0.000000


In [51]:
user = pd.merge(movies, newuser, on="movieId")

In [54]:
user

,movieId,title,genres,91
0,1,toy story,Adventure Animation Children Comedy Fantasy,91.686738
1,2,jumanji,Adventure Children Fantasy,0.000000
2,3,grumpier old men,Comedy Romance,0.000000
3,4,waiting to exhale,Comedy Drama Romance,1.255624
4,5,father of the bride part ii,Comedy,13.154490
...,...,...,...,...
8561,193579,jon stewart has left the building,Documentary,0.029516
8562,193581,black butler: book of the atlantic,Action Animation Comedy Fantasy,0.033733
8563,193583,no game no life: zero,Animation Comedy Fantasy,0.029516
8564,193585,flint,Drama,0.029516


In [58]:
user.columns = ['movieId', 'title', 'genres', 'scores']
   

In [59]:
user

,movieId,title,genres,scores
0,1,toy story,Adventure Animation Children Comedy Fantasy,91.686738
1,2,jumanji,Adventure Children Fantasy,0.000000
2,3,grumpier old men,Comedy Romance,0.000000
3,4,waiting to exhale,Comedy Drama Romance,1.255624
4,5,father of the bride part ii,Comedy,13.154490
...,...,...,...,...
8561,193579,jon stewart has left the building,Documentary,0.029516
8562,193581,black butler: book of the atlantic,Action Animation Comedy Fantasy,0.033733
8563,193583,no game no life: zero,Animation Comedy Fantasy,0.029516
8564,193585,flint,Drama,0.029516


In [70]:
user_sorted = user.sort_values(by = 'scores',ascending=False)
  

In [71]:
user_sorted

,movieId,title,genres,scores
243,296,pulp fiction,Comedy Crime Drama Thriller,145.821747
487,593,"silence of the lambs, the",Crime Horror Thriller,118.820510
212,260,star wars: episode iv - a new hope,Action Adventure Sci-Fi,114.589938
484,589,terminator 2: judgment day,Action Sci-Fi,106.047278
856,1198,raiders of the lost ark (indiana jones and the...,Action Adventure,103.143962
...,...,...,...,...
410,494,executive decision,Action Adventure Thriller,0.000000
2686,3810,white sands,Drama Thriller,0.000000
4355,7028,quick change,Comedy Crime,0.000000
415,500,mrs. doubtfire,Comedy Drama,0.000000


In [72]:
user_sorted.to_csv('user1_sorted.csv')

In [40]:
user1.to_csv('user1.csv')

In [45]:
user_final_ratings.iloc[200].sort_values(ascending = False)[0:10]
user2 = user_final_ratings.iloc[200].sort_values(ascending = False)

In [46]:
user2.head()

movieId
318     117.134606
593      98.809788
2571     98.387278
110      90.621680
2959     79.003219
Name: 201, dtype: float64

In [47]:
user2.to_csv('user2.csv')